## Using `None`

In the main session, we created the following function.

In [1]:
def greet_person(name='unknown'):
    if name == 'unknown':
        print("Hello. What's your name?")
    else:
        print("Hello,", name)

This works fine, provided someone doesn't actually have the name 'unknown'.

In [2]:
greet_person()
greet_person("Ann")
greet_person("unknown")

Hello. What's your name?
Hello, Ann
Hello. What's your name?


This may be unlikely but it's sloppy code none-the-less. Instead, it would be better for us to use `None` is the default argument. We also change the `==` to `is` (it would work the same regardless, but this way is best practice and slightly faster).

In [3]:
def greet_person(name=None):
    if name is None:
        print("Hello. What's your name?")
    else:
        print("Hello,", name)

In [4]:
greet_person()
greet_person("Ann")
greet_person("unknown")

Hello. What's your name?
Hello, Ann
Hello, unknown


It's also important to note, that whenever we reach the end of a function call without returning, Python will do an automatic return of `None` for us.

In [5]:
def no_return():
    print("Hello")
    
x = no_return()
print(x)

Hello
None


## Passing by reference/value

We saw is the additional notes for last session that when we assign a list to a new variable, we only create a new reference to the same data in memory rather than a copy of the values. The same is true for passing a list as a function argument. We combat this in the same way by using a full slice (`my_list[:]`).

In [9]:
numbers = [1, 2, 3]

def double_and_print(num_list):
    for i, n in enumerate(num_list):
        num_list[i] = 2 * n
    print(num_list)

double_and_print(numbers)
print(numbers)

[2, 4, 6]
[2, 4, 6]


In [11]:
numbers = [1, 2, 3]

def double_and_print(num_list):
    num_list = num_list[:]  # copy by value
    for i, n in enumerate(num_list):
        num_list[i] = 2 * n
    print(num_list)

double_and_print(numbers)
print(numbers)

[2, 4, 6]
[1, 2, 3]


## Args and kwargs

We've seen functions such as `print()` that can take any numbers of inputs. How do we do this with our own functions? The answer: we use splatting. The convention is to splat any additional argument into a tuple called `args` though a more contextual name is also valid..

In [14]:
def alternating_sum(*args):
    return sum(e if i % 2 == 0 else -e
               for i, e in enumerate(args))

# 1 - 2 + 3 - 4 = -2
alternating_sum(1, 2, 3, 4)

-2

We can combine this with non-splatted arguments.

In [16]:
def alternating_sum(start=0, *args):
    return start + sum(e if i % 2 == 0 else -e
                       for i, e in enumerate(args))

# 10 + (1 - 2 + 3 - 4) = 8
alternating_sum(10, 1, 2, 3, 4)

8

Note, that a splatted parameter can come after optional parameters.

`kwargs` are similar to `args` but allow us to handle named arguments. We'll learn more about this in session seven.

## Lambda functions

Lambda functions give us a one-line shorthand for simple functions, closely resembling that of mathematical notation.

In [17]:
def add_regular(x, y):
    return x + y

print(add_regular(2, 3))

5


In [21]:
add_lambda = lambda x, y: x + y

print(add_lambda(2, 3))

5


These can also use optional parameters and args/kwargs.

## Functional programming

Lambda functions are useful for performing functional programming. This is a coding paradigm in which we focus more on the operation we are performing than the data we are performing it on. There are three central techniques: mapping, filtering, and reducing.

Mapping involves using a function that maps a single input to a single output (most often defined using a lambda function) and applying it to each element of a list, tuple, etc.

In [24]:
numbers = [1, 2, 3, 4, 5]
doubled = map(lambda x: 2 * x, numbers)
print(doubled)

This gives rise to a map object. This differs from a list in that it is evaluated lazily. That is, no computations are performed until strictly necessary. We can convert this to a list with `list()`.

In [25]:
print(list(doubled))

[2, 4, 6, 8, 10]


Filtering involves using a function that maps single elements to Boolean values, and keeps only elements that map to `True`. This gives rise to a `filter` object which can likewise be converted to a list.

In [26]:
numbers = [1, -2, 3, -4, 5]
positive = filter(lambda x: x > 0, numbers)
print(list(positive))

[1, 3, 5]


Note, these above two could be performed with list comprehension, but `map`/`filter` are more efficient for large inputs/slow operations.

Finally, we have reduction. This solves a different sort of problem, such as the following.

In [27]:
numbers = [1, 2, 3, 4, 5]
product = 1
for n in numbers:
    product *= n
print(product)

120


To use the `reduce` function, we have to import it as follows.

In [28]:
from functools import reduce

`reduce` use a function which takes two inputs, the accumulator and value, and returns a single value. The accumulator takes the place of `product` above and `value` takes the place of `n`.

In [31]:
reduce(lambda acc, x: acc * x, numbers)

120

## Accessing global variables

Although we can read global variables in a local scope (provided there isn't a more local variable with the same name), we cannot modify them by default. Instead we will create a local variable or get an assignment error for using a local variable before it is defined.

In [32]:
n = 10

def n_to_twenty():
    n = 20
    
n_to_twenty()
print(n)

10


In [33]:
n = 10

def n_to_twenty():
    n = 2 * n
    
n_to_twenty()
print(n)

UnboundLocalError: local variable 'n' referenced before assignment

To get around this, we have to explicitly tell Python that we wish to use the global version of `n`.

In [35]:
n = 10

def n_to_twenty():
    global n
    n = 20
    
n_to_twenty()
print(n)

20


If `n` didn't exist in the global scope, it will be created. Since it has global scope, it will then exist after the function ends.

**This is incredibly bad practice** and can be avoided by passing `n` as a parameter. That said, some people code like this so it is important to be aware of.

## Memoisation

Take a look at our recursive Fibonacci function.

In [37]:
def fibonacci(n):
    if n == 0 or n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

This function is way slower than it should be.

In [42]:
%%timeit
fibonacci(35)

2.65 s ± 6.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The reason for this, is that we have highly overlapping sub-cases. To calculate `fibonacci(n)`, we need `fibonacci(n-1)` and `fibonacci(n-2)`. To calculate `fibonacci(n-1)` we need `fibonacci(n-2)` and `fibonacci(n-3)`. That is, we compute `fibonacci(n-2)` twice. This duplication gets worse the deeper we get to lead to near exponential growth in computation time. 

It would be better if we could remember the values we calculated and reuse them. This is known as memoisation (think 'memory'). Here are two implementations, one simple, one flexible.

In [43]:
n = 35

# `None` if we haven't calculated a value yet
mem = [None] * (n + 1)
# Base case
mem[0] = 1
mem[1] = 1

def fibonacci(n):
    if mem[n]:
        return mem[n]
    else:
        val = fibonacci(n-1) + fibonacci(n-2)
    mem[n] = val
    return val

In [45]:
%%timeit
fibonacci(35)

84.9 ns ± 0.829 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


On my PC, this is roughly ten million times faster than the above solution.

In [47]:
def fibonacci(n):
    mem = [None] * (n + 1)
    # Base case
    mem[0] = 1
    mem[1] = 1
    
    # Internal function using memory
    def _fib(n):
        if mem[n]:
            return mem[n]
        else:
            val = _fib(n-1) + _fib(n-2)
        mem[n] = val
        return val
    _fib(n)

In [48]:
%%timeit
fibonacci(35)

7.31 µs ± 16.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The latter is preferred because it's entirely self-contained. It is slightly slower but not significantly so.